In [1]:
pip install pdfplumber python-docx nltk langchain

  Using cached pdfplumber-0.11.6-py3-none-any.whl.metadata (42 kB)
  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached langchain-0.3.23-py3-none-any.whl.metadata (7.8 kB)
  Using cached pdfminer_six-20250327-py3-none-any.whl.metadata (4.1 kB)
  Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl.metadata (48 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached langchain_core-0.3.51-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.3.30-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached requests-2.3

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Sejal
[nltk_data]     Hanmante\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
pip install ipywidgets

  Using cached ipywidgets-8.1.6-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.14-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.6-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.14-py3-none-any.whl (213 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pdfplumber
import docx
import nltk
from nltk.tokenize import word_tokenize
from ipywidgets import FileUpload
import io

nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Sejal
[nltk_data]     Hanmante\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

###  Text Extraction functions

In [6]:
def extract_text_from_pdf(file_stream):
    text = ""
    with pdfplumber.open(file_stream) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def extract_text_from_docx(file_stream):
    doc = docx.Document(file_stream)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_text_from_txt(file_stream):
    return file_stream.read().decode('utf-8')

def extract_text(uploaded_file):
    filename = list(uploaded_file.value.keys())[0]
    content = uploaded_file.value[filename]['content']
    ext = filename.split('.')[-1].lower()
    file_stream = io.BytesIO(content)

    if ext == 'pdf':
        return extract_text_from_pdf(file_stream)
    elif ext == 'docx':
        return extract_text_from_docx(file_stream)
    elif ext == 'txt':
        return extract_text_from_txt(file_stream)
    else:
        raise ValueError("Unsupported file type. Please upload a PDF, DOCX, or TXT file.")


### Chunking function

In [7]:
def chunk_text(text, chunk_size=200, overlap=50):
    words = word_tokenize(text)
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks


In [20]:
upload_widget = FileUpload(accept='.pdf,.docx,.txt', multiple=False)
display(upload_widget)


FileUpload(value=(), accept='.pdf,.docx,.txt', description='Upload')

In [ ]:
"""import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
"""

[nltk_data] Downloading package punkt to C:\Users\Sejal
[nltk_data]     Hanmante\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sejal
[nltk_data]     Hanmante\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sejal
[nltk_data]     Hanmante\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Sejal
[nltk_data]     Hanmante\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [27]:

from nltk.tokenize import word_tokenize
from ipywidgets import FileUpload, Output, VBox


# ✅ Define text extractors
def extract_text_from_pdf(file_stream):
    with pdfplumber.open(file_stream) as pdf:
        return '\n'.join(page.extract_text() for page in pdf.pages if page.extract_text())

def extract_text_from_docx(file_stream):
    doc = docx.Document(file_stream)
    return '\n'.join([para.text for para in doc.paragraphs])

def extract_text_from_txt(file_stream):
    return file_stream.read().decode('utf-8')

# ✅ General extractor
def extract_text(uploaded_file):
    file_info = uploaded_file.value[0]
    filename = file_info['name']
    content = file_info['content']
    ext = filename.split('.')[-1].lower()
    file_stream = io.BytesIO(content)

    if ext == 'pdf':
        return extract_text_from_pdf(file_stream)
    elif ext == 'docx':
        return extract_text_from_docx(file_stream)
    elif ext == 'txt':
        return extract_text_from_txt(file_stream)
    else:
        raise ValueError("Unsupported file type")

# ✅ Text chunker
def chunk_text(text, chunk_size=200, overlap=50):
    words = word_tokenize(text)
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = words[start:end]
        chunks.append(' '.join(chunk))
        start += chunk_size - overlap
    return chunks

# ✅ Output box
output_box = Output()

# ✅ Function to run on file upload
def on_file_upload(change):
    output_box.clear_output()
    with output_box:
        try:
            if upload_widget.value:
                raw_text = extract_text(upload_widget)
                chunks = chunk_text(raw_text)

                print(f"✅ Total Chunks: {len(chunks)}")
                for i, chunk in enumerate(chunks[:5]):
                    print(f"\n--- Chunk {i+1} ---\n{chunk[:500]}...")
            else:
                print("⚠️ No file uploaded.")
        except Exception as e:
            print(f"❌ Error: {e}")

# ✅ Upload widget
upload_widget = FileUpload(accept='.pdf,.docx,.txt', multiple=False)
upload_widget.observe(on_file_upload, names='value')

# ✅ Show UI
display(VBox([upload_widget, output_box]))

# ✅ Define text extractors
def extract_text_from_pdf(file_stream):
    with pdfplumber.open(file_stream) as pdf:
        return '\n'.join(page.extract_text() for page in pdf.pages if page.extract_text())

def extract_text_from_docx(file_stream):
    doc = docx.Document(file_stream)
    return '\n'.join([para.text for para in doc.paragraphs])

def extract_text_from_txt(file_stream):
    return file_stream.read().decode('utf-8')

# ✅ General extractor
def extract_text(uploaded_file):
    file_info = uploaded_file.value[0]
    filename = file_info['name']
    content = file_info['content']
    ext = filename.split('.')[-1].lower()
    file_stream = io.BytesIO(content)

    if ext == 'pdf':
        return extract_text_from_pdf(file_stream)
    elif ext == 'docx':
        return extract_text_from_docx(file_stream)
    elif ext == 'txt':
        return extract_text_from_txt(file_stream)
    else:
        raise ValueError("Unsupported file type")

# ✅ Text chunker
def chunk_text(text, chunk_size=200, overlap=50):
    words = word_tokenize(text)
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = words[start:end]
        chunks.append(' '.join(chunk))
        start += chunk_size - overlap
    return chunks

# ✅ Output box
output_box = Output()

# ✅ Function to run on file upload
def on_file_upload(change):
    output_box.clear_output()
    with output_box:
        try:
            if upload_widget.value:
                raw_text = extract_text(upload_widget)
                chunks = chunk_text(raw_text)

                print(f"✅ Total Chunks: {len(chunks)}")
                for i, chunk in enumerate(chunks[:5]):
                    print(f"\n--- Chunk {i+1} ---\n{chunk[:500]}...")
            else:
                print("⚠️ No file uploaded.")
        except Exception as e:
            print(f"❌ Error: {e}")

# ✅ Upload widget
upload_widget = FileUpload(accept='.pdf,.docx,.txt', multiple=False)
upload_widget.observe(on_file_upload, names='value')

# ✅ Show UI
display(VBox([upload_widget, output_box]))


In [24]:
# 🔁 Call this manually after uploading
process_uploaded_file()

⚠️ Please upload a file.


In [ ]:
"""pip install transformers"""

  Using cached transformers-4.51.2-py3-none-any.whl.metadata (38 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
Using cached transformers-4.51.2-py3-none-any.whl (10.4 MB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.6 MB 3.4 MB/s eta 0:00:04
   ----- ---------------------------------- 1.8/12.6 MB 5.0 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/12.6 MB 5.8 MB/s eta 0:00:02
   --------------- ------------------------ 5.0/12.6 MB 6.6 MB/s eta 0:00:02
   --------------------- ------------------ 6.8/12.6 MB 7.0 MB/s eta 0:00:01
 

In [ ]:
"""pip install torch """

  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
Using cached torch-2.6.0-cp312-cp312-win_amd64.whl (204.1 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl (15 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import pdfplumber
import re
import nltk
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

### InsuranceBERT model

In [ ]:
# Load InsuranceBERT model
# Load model directly
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("llmware/industry-bert-insurance-v0.1")
model = AutoModel.from_pretrained("llmware/industry-bert-insurance-v0.1")

def get_embedding(text):
    tokens = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        output = model(**tokens)
    embeddings = output.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()


RuntimeError: Failed to import transformers.models.timm_wrapper.configuration_timm_wrapper because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
pip uninstall timm


### Preprocessing functions

In [ ]:
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

def clean_text(text):
    text = re.sub(r'\n+', ' ', text)  # remove newlines
    text = re.sub(r'\s+', ' ', text)  # normalize whitespace
    text = re.sub(r'Page\s*\d+|\d+\s*/\s*\d+', '', text)  # remove common page numbers
    return text.strip()

def chunk_text(text, chunk_size=200, overlap=50):
    words = word_tokenize(text)
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks


### Processing all pdfs in the folder 


In [ ]:
pdf_folder = r"D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\policy_pdfs"
all_chunks = []
all_embeddings = []
metadata = []

for filename in tqdm(os.listdir(pdf_folder)):
    if filename.endswith(".pdf"):
        path = os.path.join(pdf_folder, filename)
        raw_text = extract_text_from_pdf(path)
        cleaned_text = clean_text(raw_text)
        chunks = chunk_text(cleaned_text)

        for i, chunk in enumerate(chunks):
            embedding = get_embedding(chunk)
            all_chunks.append(chunk)
            all_embeddings.append(embedding)
            metadata.append({"file": filename, "chunk_id": i})


In [ ]:
import numpy as np
import pickle

# Save using pickle
with open("embeddings.pkl", "wb") as f:
    pickle.dump({
        "chunks": all_chunks,
        "embeddings": all_embeddings,
        "metadata": metadata
    }, f)

print("✅ Embeddings and chunks saved.")


### FAISS INDEXING

In [ ]:
import faiss
import numpy as np
import pickle


In [ ]:
# Load embeddings, chunks, metadata
with open("embeddings.pkl", "rb") as f:
    data = pickle.load(f)

all_embeddings = np.array(data["embeddings"]).astype("float32")
all_chunks = data["chunks"]
metadata = data["metadata"]


In [ ]:
# Create FAISS index
dimension = all_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(all_embeddings)

# Save FAISS index
faiss.write_index(index, "policy_index_docs.faiss")

# Save metadata and chunks for lookup
with open("policy_metadata.pkl", "wb") as f:
    pickle.dump({"chunks": all_chunks, "metadata": metadata}, f)

print("✅ FAISS index and metadata saved.")


In [ ]:
# Load FAISS index
index = faiss.read_index("policy_index_docs.faiss")

# Load metadata & chunks
with open("policy_metadata.pkl", "rb") as f:
    db = pickle.load(f)

chunks = db["chunks"]
metadata = db["metadata"]


In [ ]:
def search_policy(query, k=3):
    query_embedding = get_embedding(query).astype("float32").reshape(1, -1)
    distances, indices = index.search(query_embedding, k)

    results = []
    for i in indices[0]:
        results.append({
            "chunk": chunks[i],
            "file": metadata[i]["file"],
            "chunk_id": metadata[i]["chunk_id"]
        })
    return results


### Query Handling

In [ ]:
import faiss
import pickle
import numpy as np

# Load FAISS index
index = faiss.read_index("policy_index.faiss")

# Load metadata
with open("policy_metadata.pkl", "rb") as f:
    db = pickle.load(f)

chunks = db["chunks"]
metadata = db["metadata"]


### Query Search + prompt builder 

In [ ]:
def retrieve_relevant_chunks(query, k=3):
    query_embedding = get_embedding(query).astype("float32").reshape(1, -1)
    distances, indices = index.search(query_embedding, k)

    results = []
    for i in indices[0]:
        results.append({
            "chunk": chunks[i],
            "file": metadata[i]["file"],
            "chunk_id": metadata[i]["chunk_id"]
        })
    return results

def build_prompt(retrieved_chunks, query):
    context = "\n\n".join([f"[{r['file']} - Chunk {r['chunk_id']}]:\n{r['chunk']}" for r in retrieved_chunks])
    prompt = f"""You are a helpful assistant. Answer the user's question based only on the provided policy content.

Context:
{context}

Question: {query}
"""
    return prompt


In [ ]:
import openai
openai.api_base = "https://openrouter.ai/api/v1"
openai.api_key = "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"

def ask_deepseek(prompt, model="deepseek/deepseek-r1:free"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant for insurance policy queries."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=800,
    )
    return response.choices[0].message['content']


In [ ]:
query = "Does this policy provide cashless hospitalization for critical illness?"

# Step 1: Retrieve
top_chunks = retrieve_relevant_chunks(query)

# Step 2: Build Prompt
prompt = build_prompt(top_chunks, query)

# Step 3: Ask LLM
answer = ask_deepseek(prompt)

# Step 4: Display
print("📄 Retrieved Context:")
for i, chunk in enumerate(top_chunks):
    print(f"\n[{chunk['file']} - Chunk {chunk['chunk_id']}]:\n{chunk['chunk'][:400]}...\n")

print("\n🧠 Answer:")
print(answer)
